#Multiple Features Based Approach for Automatic Fake News Detection<br>
**Team:**<br>
**181CO204**-Akshat Nambiar<br>
**181CO232**-Mohammed Rushad

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import io
import matplotlib
from subprocess import check_output
from nltk import PorterStemmer

In [2]:
from google.colab import files
uploaded = files.upload()

Saving fake3.csv to fake3.csv


In [3]:
all_news = pd.read_csv('/content/fake3.csv')

#1. The types of labels in our reduced dataset:

In [4]:
print("Types of stories", all_news.groupby(["type"]).size())

Types of stories type
bias            56
bs            1726
conspiracy     117
fake             3
hate            60
satire          38
dtype: int64


In [5]:
all_news.head()

,Unnamed: 0,Unnamed: 0.1,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,0,3406,cd1f3c1ff3a58871c1c506cb8a9ef770a94e380a,0,Tom Winter (noreply@blogger.com),2016-11-01T23:18:18.586+02:00,“Stop watching RT and other Russian-biased sou...,"November 1, 2016 - \nBy Tom Winter exclusivel...",english,2016-11-01T23:18:18.586+02:00,fort-russ.com,US,NaN,“Stop watching RT and other Russian-biased sou...,0.000,NaN,0,1,0,0,0,bs
1,1,757,d62100eee1b1cb32b8d3a599a727f168ca11276e,0,hqanon,2016-10-27T03:00:00.000+03:00,"NATO, US, & UK To Assemble Largest Troop Build...",By: John Vibes via thefreethoughtproject.com\n...,english,2016-10-28T01:13:00.968+03:00,anonhq.com,US,12993.0,"NATO, US, & UK To Assemble Largest Troop Build...",0.966,http://anonhq.com/wp-content/uploads/2016/10/F...,2,3,0,0,0,bs
2,2,3624,b687c1a33b4f53c865c7c4e824b9fcc099f04030,0,US Marine Fighting Tyranny,2016-10-27T23:36:25.030+03:00,Russia warships: Kuznetsov battle group ‘refue...,BBC News A group of warships including Russia’...,english,2016-10-27T23:36:25.030+03:00,fromthetrenchesworldreport.com,US,NaN,Russia warships: Kuznetsov battle group ‘refue...,0.000,http://www.fromthetrenchesworldreport.com/wp-c...,0,1,0,0,0,bs
3,3,4544,0ce8687f7cb8df0b853033f009b25a343605d3e8,0,Jameson Parker,2016-11-03T22:17:00.000+02:00,Love Trumps Hate: GoFundMe To Rebuild Church T...,Google Pinterest Digg Linkedin Reddit Stumbleu...,english,2016-11-04T12:32:18.347+02:00,ifyouonlynews.com,US,17592.0,Love Trumps Hate: GoFundMe To Rebuild Church T...,0.000,http://www.ifyouonlynews.com/wp-content/upload...,0,1,0,0,0,bs
4,4,3235,d6c5e328c9f70645ceb752a909ca444da8e72afd,0,Auntie Lulu,2016-11-03T17:14:50.777+02:00,Comment on Migrants battle on the streets of P...,"Posted on November 3, 2016 by DCG | 3 Comments...",english,2016-11-03T17:14:50.777+02:00,fellowshipoftheminds.com,US,NaN,Comment on Migrants battle on the streets of P...,0.000,https://fellowshipofminds.files.wordpress.com/...,0,1,0,0,0,bs


#2. Re-mapping labels to fake and real news:

In [6]:
all_news["type"]= all_news["type"].replace("bs","fake")
all_news["type"]= all_news["type"].replace("conspiracy","fake")

all_news["type"]= all_news["type"].replace("satire","real")
all_news["type"]= all_news["type"].replace("bias","real")
all_news["type"]= all_news["type"].replace("hate","real")
all_news["type"]= all_news["type"].replace("junksci","real")
all_news["type"]= all_news["type"].replace("state","real")

all_news.type = all_news.type.map(dict(real=1, fake=0))

In [7]:
all_news["type"]

0       0
1       0
2       0
3       0
4       0
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Name: type, Length: 2000, dtype: int64

#3. Cleaning data and concatenating title and text:

In [8]:
import nltk
nltk.download('punkt')
# Fill any blank fields
all_news.title.fillna("", inplace=True)
all_news.text.fillna("", inplace=True)
all_news.author.fillna("", inplace=True)
all_news.country.fillna("", inplace=True)
all_news.type.fillna(all_news.type.mean())
all_news.participants_count.fillna(all_news.participants_count.mean())
all_news.domain_rank.fillna(all_news.domain_rank.mean())
all_news.spam_score.fillna(all_news.spam_score.mean())
all_news.replies_count.fillna(all_news.replies_count.mean())
all_news.likes.fillna(all_news.likes.mean())
all_news.comments.fillna(all_news.comments.mean())
all_news.shares.fillna(all_news.shares.mean())
all_news.site_url.fillna("", inplace=True)


# Join the title and text
all_text = all_news.title.str.cat(all_news.text, sep=' ')

all_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0       “Stop watching RT and other Russian-biased sou...
1       NATO, US, & UK To Assemble Largest Troop Build...
2       Russia warships: Kuznetsov battle group ‘refue...
3       Love Trumps Hate: GoFundMe To Rebuild Church T...
4       Comment on Migrants battle on the streets of P...
                              ...                        
1995    Saudi Arabia announces date certain for the ap...
1996    Hoping for the best and preparing for the wors...
1997    Katheryn From My Walk & Talk Katheryn From My ...
1998    Black Turnout Soft in Early Voting, Boding Ill...
1999    Destruction of US Abrams Tank: Mosul Offensive...
Name: title, Length: 2000, dtype: object

#4. NLP-Preprocessing and Vectorization of text data and combining all features:

##4.1 Processing title+text features:

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [10]:
cvec = CountVectorizer()
X = all_text
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train = pd.DataFrame(cvec.transform(X_train).todense(), 
                        columns=cvec.get_feature_names())

In [12]:
df_test = pd.DataFrame(cvec.transform(X_test).todense(),
                       columns=cvec.get_feature_names())

In [13]:
print(df_train.shape)
print(y_train.shape)
print(df_test.shape)
print(y_test.shape)

(1400, 40135)
(1400,)
(600, 40135)
(600,)


In [14]:
lr = LogisticRegression(max_iter=1000)
lr.fit(df_train, y_train)
lr.score(df_test,y_test)

0.925

##4.2 Processing title+text, author:

In [15]:
X = all_news.author
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [16]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
author_train = pd.DataFrame(cvec.transform(X_train).todense(),
                                  columns=cvec.get_feature_names())
author_test = pd.DataFrame(cvec.transform(X_test).todense(),
                                 columns=cvec.get_feature_names())

In [17]:
train = pd.concat([df_train, author_train], axis=1)
test = pd.concat([df_test, author_test], axis=1)

In [18]:
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)

(1400, 40965)
(1400,)
(600, 40965)
(600,)


In [19]:
lr = LogisticRegression(max_iter=1000)
lr.fit(train, y_train)
lr.score(test, y_test)

0.9083333333333333

##4.3 Processing data with Participants Count feature:

In [20]:
X = all_news.participants_count
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [21]:
participants_train = X_train
participants_test = X_test

In [22]:
participants_test.reset_index(drop=True, inplace=True)
participants_train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [23]:
train = pd.concat([participants_train, df_train, author_train], axis=1)
test = pd.concat([participants_test, df_test, author_test], axis=1)

In [24]:
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)

(1400, 40966)
(1400,)
(600, 40966)
(600,)


In [25]:
lr = LogisticRegression(max_iter=800)
lr.fit(train, y_train)
lr.score(test, y_test)

0.9016666666666666

##4.4 Processing data with Country feature:

In [26]:
X = all_news.country
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [27]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
country_train = pd.DataFrame(cvec.transform(X_train).todense(),
                                  columns=cvec.get_feature_names())
country_test = pd.DataFrame(cvec.transform(X_test).todense(),
                                 columns=cvec.get_feature_names())

In [28]:
train = pd.concat([country_train, participants_train, df_train, author_train], axis=1)
test = pd.concat([country_test, participants_test, df_test, author_test], axis=1)

In [29]:
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)

(1400, 40976)
(1400,)
(600, 40976)
(600,)


In [30]:
lr = LogisticRegression(max_iter=800)
lr.fit(train, y_train)
lr.score(test, y_test)

0.895

##4.5 Processing data with remaining numerical features:

In [31]:
X = pd.concat([all_news.spam_score, all_news.replies_count, all_news.likes, all_news.comments, all_news.shares],axis=1)
X.shape

(2000, 5)

In [32]:
X

,spam_score,replies_count,likes,comments,shares
0,0.000,0,0,0,0
1,0.966,2,0,0,0
2,0.000,0,0,0,0
3,0.000,0,0,0,0
4,0.000,0,0,0,0
...,...,...,...,...,...
1995,0.000,0,0,0,0
1996,0.000,0,0,0,0
1997,0.117,0,0,0,0
1998,0.000,0,0,0,0


In [33]:
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [34]:
X_train.head()

,spam_score,replies_count,likes,comments,shares
1244,0.0,0,0,0,0
1522,0.0,0,0,0,0
908,0.0,0,0,0,0
1207,0.0,0,0,0,0
389,0.0,0,0,0,0


In [35]:
rest_train = X_train
rest_test = X_test

In [36]:
rest_test.reset_index(drop=True, inplace=True)
rest_train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [37]:
rest_train.head()

,spam_score,replies_count,likes,comments,shares
0,0.0,0,0,0,0
1,0.0,0,0,0,0
2,0.0,0,0,0,0
3,0.0,0,0,0,0
4,0.0,0,0,0,0


In [38]:
train = pd.concat([rest_train, country_train, participants_train, df_train, author_train], axis=1)
test = pd.concat([rest_test, country_test, participants_test, df_test, author_test], axis=1)

In [39]:
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)

(1400, 40981)
(1400,)
(600, 40981)
(600,)


In [40]:
lr = LogisticRegression(max_iter=800)
lr.fit(train, y_train)
lr.score(test, y_test)

0.9116666666666666

##4.6 Processing data with Site URL feature to create final dataset:

In [41]:
all_news.site_url

0                        fort-russ.com
1                           anonhq.com
2       fromthetrenchesworldreport.com
3                    ifyouonlynews.com
4             fellowshipoftheminds.com
                     ...              
1995                       awdnews.com
1996    fromthetrenchesworldreport.com
1997            dennismichaellynch.com
1998                         amren.com
1999                  activistpost.com
Name: site_url, Length: 2000, dtype: object

In [42]:
X = all_news.site_url
y = all_news.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [43]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
url_train = pd.DataFrame(cvec.transform(X_train).todense(),
                                  columns=cvec.get_feature_names())
url_test = pd.DataFrame(cvec.transform(X_test).todense(),
                                 columns=cvec.get_feature_names())

In [44]:
train = pd.concat([url_train, rest_train, country_train, participants_train, df_train, author_train], axis=1)
test = pd.concat([url_test, rest_test, country_test, participants_test, df_test, author_test], axis=1)

In [45]:
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)

(1400, 41075)
(1400,)
(600, 41075)
(600,)


#5. Testing Models on final data:

##5.1 Logistic Regression

In [46]:
lr = LogisticRegression(max_iter=800)
lr.fit(train, y_train)
print("Accuracy for Logistic Regression:")
lr.score(test, y_test)

Accuracy for Logistic Regression:


0.91

In [47]:
from sklearn.metrics import classification_report
y_pred = lr.predict(test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       552
           1       0.25      0.06      0.10        48

    accuracy                           0.91       600
   macro avg       0.59      0.52      0.53       600
weighted avg       0.87      0.91      0.88       600



##5.2 K Nearest Neighbours (KNN)

In [48]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(train, y_train)
print("Accuracy for KNN:")
model.score(test, y_test)


Accuracy for KNN:


0.915

In [49]:
y_pred = model.predict(test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96       552
           1       0.29      0.04      0.07        48

    accuracy                           0.92       600
   macro avg       0.60      0.52      0.51       600
weighted avg       0.87      0.92      0.88       600



##5.3 Support Vector Machine (SVM)

In [50]:
#SVM
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(train, y_train)
print("Accuracy for SVM:")
clf.score(test, y_test)

Accuracy for SVM:


0.9

In [51]:
y_pred = clf.predict(test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95       552
           1       0.32      0.23      0.27        48

    accuracy                           0.90       600
   macro avg       0.63      0.59      0.61       600
weighted avg       0.89      0.90      0.89       600



##5.4 Decision Tree (DT)

In [52]:
#DecisionTree: gini
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier()
clf1 = clf1.fit(train,y_train)
print("Accuracy for Decision Tree using gini Index:")
clf1.score(test, y_test)

Accuracy for Decision Tree using gini Index:


1.0

In [53]:
y_pred = clf1.predict(test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       552
           1       1.00      1.00      1.00        48

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



##5.5 Naive Bayes

In [54]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
model1 = GaussianNB()
model1.fit(train, y_train)
print("Accuracy for Naive Bayes:")
model1.score(test, y_test)

Accuracy for Naive Bayes:


0.92

In [55]:
y_pred = model1.predict(test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       552
           1       0.50      0.08      0.14        48

    accuracy                           0.92       600
   macro avg       0.71      0.54      0.55       600
weighted avg       0.89      0.92      0.89       600



##5.6 Long Short-Term Memory (LSTM)

In [56]:
train1 = train.to_numpy()
test1 = test.to_numpy()

In [57]:
#LSTM
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(7)
top_words = 5000
max_review_length = 500
train1
train1 = sequence.pad_sequences(train1, maxlen=max_review_length)
test1 = sequence.pad_sequences(test1, maxlen=max_review_length)

#Making the model
embedding_vecor_length = 32
model2 = Sequential()
model2.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(train1, y_train, epochs=5, batch_size=64)


scores = model2.evaluate(test1, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
22/22 [==============================] - 12s 473ms/step - loss: 0.5622 - accuracy: 0.7828
Epoch 2/5
22/22 [==============================] - 10s 470ms/step - loss: 0.2628 - accuracy: 0.9283
Epoch 3/5
22/22 [==============================] - 10s 475ms/step - loss: 0.2770 - accuracy: 0.9208
Epoch 4/5
22/22 [==============================] - 10s 473ms/step - loss: 0